In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import *

## Simple Custom Dense Layer

In [2]:
class SimpleDense(layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel', initial_value=w_init(shape=(input_shape[-1], self.units),
                             dtype='float32'),trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias', initial_value=b_init(shape=self.units, dtype='float32'), trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

### Basic Demo

In [3]:
inputs = tf.constant([[5]], dtype='float32')
inputs

Metal device set to: Apple M1


2022-01-30 12:06:23.446238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-30 12:06:23.446323: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[5.]], dtype=float32)>

In [4]:
custom_dense = SimpleDense(units=1)

In [5]:
custom_dense(inputs)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.4105938]], dtype=float32)>

In [6]:
custom_dense.variables

[<tf.Variable 'custom_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.08211876]], dtype=float32)>,
 <tf.Variable 'custom_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

## Load and Preprocess Data

In [7]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [8]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

## Training and Evaluation with Simple Dense

In [9]:
model = Sequential([layers.Input(shape=(28, 28)),
                    layers.Flatten(),
                    SimpleDense(128),
                    layers.Lambda(lambda x : tf.maximum(x, 0.0)),
                    layers.Dropout(0.1),
                    SimpleDense(64),
                    layers.Lambda(lambda x : tf.maximum(x, 0.0)),
                    layers.Dropout(0.1),
                    layers.Dense(10, activation='softmax')])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 custom_dense_1 (CustomDense  (None, 128)              100480    
 )                                                               
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_dense_2 (CustomDense  (None, 64)               8256      
 )                                                               
                                                                 
 lambda_1 (Lambda)           (None, 64)                0

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='acc')
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

  23/1875 [..............................] - ETA: 8s - loss: 0.1339 - acc: 0.9579 

2022-01-30 12:15:37.534218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1869/1875 [============================>.] - ETA: 0s - loss: 0.1290 - acc: 0.9603

2022-01-30 12:15:45.647341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 9s 5ms/step - loss: 0.1291 - acc: 0.9603 - val_loss: 0.0968 - val_acc: 0.9694
